# Data Preparation

Prepare historical stock prices datasets for demo analysis.

## Social Media Stock Prices

Major social media historical stock prices from 2012-2022 for meta, twitter, snap, pinterest, etsy.

**Source**: [Kaggle: Major social media historical stock prices](https://www.kaggle.com/datasets/prasertk/social-media-stock-prices)

Sample:
```csv
Date,Symbol,Adj Close,Close,High,Low,Open,Volume
2012-05-18,FB,38.22999954223633,38.22999954223633,45.0,38.0,42.04999923706055,573576400.0
2012-05-21,FB,34.029998779296875,34.029998779296875,36.65999984741211,33.0,36.529998779296875,168192700.0
2012-05-22,FB,31.0,31.0,33.59000015258789,30.940000534057617,32.61000061035156,101786600.0
2012-05-23,FB,32.0,32.0,32.5,31.360000610351562,31.3700008392334,73600000.0
2012-05-24,FB,33.029998779296875,33.029998779296875,33.209999084472656,31.770000457763672,32.95000076293945,50237200.0
2012-05-25,FB,31.90999984741211,31.90999984741211,32.95000076293945,31.110000610351562,32.900001525878906,37149800.0
```

### Clean up Source Dataset

Load and cleanup the original dataset:
- Massage data types
- Rename columns
- Add additional metadata columns

Save the result back to a new `csv` file: 

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime, date


# load social media stock prices data file
orig_social_media_stock_prices = r"./data/social_media_stocks_2012-2022.csv"
df = pd.read_csv(
    orig_social_media_stock_prices,
    header=0,
    parse_dates=[0],
    date_format='%Y-%m-%d',
    on_bad_lines='skip',
)

# rename columns
[df.rename(columns={col_name: str(col_name).lower().replace(' ', '_')}, inplace=True) for col_name in list(df.columns)]
df.rename(columns={'symbol': 'ticker'}, inplace=True, errors='ignore')
# optimize datatypes and columns for performance
for col_name in ('adj_close', 'close', 'high', 'low', 'open'):
    # df[col_name] = pd.to_numeric(df[col_name].map(lambda x: round(x, ndigits=6)), downcast='float')
    df[col_name] = df[col_name].map(lambda x: round(x, ndigits=6))
# downcast volume
df['volume'] = pd.to_numeric(df['volume'], downcast='unsigned')
# add a year column
df.insert(1, 'year', pd.to_numeric(df['date'].map(lambda x: x.year), downcast='unsigned'))

print(f"df shape: {df.shape}")
# print(df.dtypes)
display(df.sample(n=10))

# save back to a csv file
output_file = r"./data/social_media_stocks_2012-2022.clean.csv"
df.to_csv(output_file, index=False)

df shape: (8193, 9)


,date,year,ticker,adj_close,close,high,low,open,volume
7375,2021-06-16,2021,PINS,69.660004,69.660004,71.599998,68.699997,70.540001,11123000
425,2013-12-17,2013,TWTR,56.450001,56.450001,57.380001,54.619999,56.970001,22115200
5514,2019-12-23,2019,TWTR,32.430000,32.430000,32.799999,31.990000,32.130001,13983300
5823,2020-03-24,2020,FB,160.979996,160.979996,161.309998,152.570007,155.210007,30440400
7403,2021-06-24,2021,FB,343.179993,343.179993,344.899994,341.790009,341.850006,12329100
4753,2019-05-17,2019,FB,185.300003,185.300003,187.580002,184.279999,184.839996,10485400
2810,2017-06-19,2017,TWTR,17.059999,17.059999,17.080000,16.750000,16.770000,11318900
1342,2015-08-14,2015,FB,94.419998,94.419998,94.720001,93.209999,93.540001,15929700
5069,2019-08-16,2019,TWTR,40.580002,40.580002,40.650002,40.110001,40.299999,7998800
6170,2020-07-01,2020,PINS,23.280001,23.280001,23.600000,22.459999,22.530001,15591300


Plot and visualize the original stock values over time

In [7]:
import plotly.graph_objs as go
import plotly.io as pio

# Plotting the wave and amps using Plotly
pio.templates.default = 'plotly_dark'
fig = go.Figure()

# iterate through tickers and graphs each with line charts
tickers = list(df['ticker'].unique())
for ticker in tickers:      
    # Add the wave trace
    xdf = df[df['ticker'] == ticker][['date', 'ticker', 'adj_close']]
    fig.add_trace(go.Scatter(x=xdf['date'], y=xdf['adj_close'], mode='lines', name=ticker))

fig.show()

### Simulate Portfolio

This cell simulate a stock trading app portfolio:
- Starts with a set amount of cash reserves in the bank
- Trades stocks daily based on the ticker value
- Adds daily trade columns: trade_price, trade_value, current_shares
- Adjusts the total portfolio value and bank cash balance

In [8]:
import pandas as pd
import numpy as np
import math
from utils import generate_varying_amplitude_wave

# read the cleaned up file
clean_stock_prices_filepath = r"./data/social_media_stocks_2012-2022.clean.csv"
df = pd.read_csv(
    clean_stock_prices_filepath,
    header=0,
    parse_dates=[0],
    date_format='%Y-%m-%d',
    on_bad_lines='skip',
)

print(f"read source file. records: {len(df)}")
display(df.head())

# Initialize starting parameters
start_shares = {'FB': 100, 'TWTR': 200, 'PINS': 100, 'SNAP': 150, 'ETSY': 50}
initial_cash_reserve = 50000.00  # Starting cash reserve for the portfolio
scale = 1.0

dfs = []
# go through each ticker and 
#   - add a sinusoidal wave for the current_shares held in the portfolio
#   - then add daily trades based on changes in current shares
#   - add trade_prices, daily profit, and portfolio share amount
for ticker in df['ticker'].unique():
    xdf = df[df['ticker'] == ticker].copy().reset_index()
    print(f"ticker: {ticker} (len: {len(xdf)})")
    starting_shares = start_shares[ticker]
    # generate a sinusoidal wave with peaks between 1.5-2.0x starting shares
    wave = generate_varying_amplitude_wave(
        length=len(xdf),
        max_amp=int(starting_shares * (1 + np.random.uniform(0.5, 1, size=1)[0])), 
        frequency=2,
        periods=3,
    )
    # shift the wave up by number of starting shares

    current_shares = pd.Series(wave + starting_shares).map(lambda x: round(x, 4)) * scale
    xdf['current_shares'] = current_shares
    # apply daily trades
    xdf['daily_trades'] = current_shares.diff().fillna(0).map(lambda x: round(x, 4))
    # pick a random tarde value
    xdf['trade_price'] = xdf.apply(lambda r: round(np.random.uniform(low=r['low'], high=r['high'], size=1)[0], ndigits=4), axis='columns')
    # add daily tarde value in $$$
    xdf['trade_value'] = -1 * round(xdf['daily_trades'] * xdf['trade_price'], ndigits=4)
    dfs.append(xdf)

# concatenate tickers dataframes together and sort
df = pd.concat(dfs, ignore_index=True)
df = df.sort_values(by=['date', 'ticker'], ignore_index=True)
# display(df.sample(n=200))

# calculating cash reserves and portfolio value
current_portfolio_value = 0.0
current_cash_reserve = initial_cash_reserve
df['cash_reserves'] = initial_cash_reserve
df['portfolio_value'] = 0.0
cur_row = 0
current_progress_percentage = 0
portfolio = {}
print(f"Processing cash reserves & portfolio values", end='')
for i, row in df.iterrows():
    df.at[i, 'cash_reserves'] = current_cash_reserve + row['trade_value']
    portfolio[row['ticker']] = round(row['current_shares'] * row['adj_close'], ndigits=4)
    # get total portfolio value
    df.at[i, 'portfolio_value'] = round(sum(portfolio.values()), ndigits=4)
    # calculate percentage
    tmp_percentage = math.floor((i + 1) / len(df) * 10)
    if current_progress_percentage != tmp_percentage:
        print('.', end='', flush=True)
        current_progress_percentage = tmp_percentage
print()
# round up cash reserves and portfolio values
df['cash_reserves'] = df['cash_reserves'].map(lambda x: round(x, 2))
df['portfolio_value'] = df['portfolio_value'].map(lambda x: round(x, 2))
# drop index column
df = df.drop(columns=['index'], errors='ignore')
# output to file
output_file = r"./data/social_media_stocks_2012-2022.final.csv"
df.to_csv(output_file, index=False)
print(f"Generation complete: ")
display(df.sample(n=20))

# print("\nchecking null values:")
# df.isna().sum()


read source file. records: 8193


,date,year,ticker,adj_close,close,high,low,open,volume
0,2012-05-18,2012,FB,38.230000,38.230000,45.000000,38.000000,42.049999,573576400
1,2012-05-21,2012,FB,34.029999,34.029999,36.660000,33.000000,36.529999,168192700
2,2012-05-22,2012,FB,31.000000,31.000000,33.590000,30.940001,32.610001,101786600
3,2012-05-23,2012,FB,32.000000,32.000000,32.500000,31.360001,31.370001,73600000
4,2012-05-24,2012,FB,33.029999,33.029999,33.209999,31.770000,32.950001,50237200


ticker: FB (len: 2447)
ticker: TWTR (len: 2077)
ticker: ETSY (len: 1717)
ticker: SNAP (len: 1244)
ticker: PINS (len: 708)
Processing cash reserves & portfolio values..........
Generation complete: 


,date,year,ticker,adj_close,close,high,low,open,volume,current_shares,daily_trades,trade_price,trade_value,cash_reserves,portfolio_value
2909,2017-07-25,2017,ETSY,14.650000,14.650000,14.685000,14.290000,14.530000,648200,50.5822,0.5822,14.3057,-8.3288,49991.67,25199.14
6798,2020-12-30,2020,ETSY,183.179993,183.179993,183.410004,176.119995,178.300003,2125600,45.7873,-0.5255,178.3346,93.7148,50093.71,24990.04
3148,2017-10-17,2017,TWTR,18.280001,18.280001,18.469999,18.110001,18.309999,8935400,210.1408,-0.2070,18.3672,3.8020,50003.80,24828.12
6667,2020-11-19,2020,TWTR,43.619999,43.619999,43.869999,42.799999,43.119999,13874000,173.4477,-0.6185,42.9873,26.5876,50026.59,25136.14
3711,2018-05-10,2018,SNAP,11.010000,11.010000,11.180000,10.930000,11.010000,19069100,11.8561,-0.3605,10.9663,3.9534,50003.95,21640.54
4013,2018-08-28,2018,ETSY,47.880001,47.880001,48.720001,46.759998,48.630001,2041800,55.2311,-0.5790,47.1516,27.3008,50027.30,21910.38
3028,2017-09-05,2017,TWTR,16.650000,16.650000,16.900000,16.440001,16.820000,10503800,215.9146,-0.1744,16.8899,2.9456,50002.95,24594.93
7540,2021-08-03,2021,PINS,60.009998,60.009998,60.049999,57.119999,57.169998,24964900,126.1351,-2.0627,58.6169,120.9091,50120.91,13490.43
1473,2015-10-15,2015,TWTR,29.709999,29.709999,30.280001,29.010000,29.459999,15838900,-94.1324,-0.8107,29.7585,24.1252,50024.13,8788.63
3914,2018-07-24,2018,FB,214.669998,214.669998,216.199997,212.600006,215.110001,28468700,90.5304,0.1086,215.1624,-23.3666,49976.63,25244.81


Visualize daily shares using plotly

In [9]:
import plotly.graph_objs as go
import plotly.io as pio

# Plotting the wave and amps using Plotly
pio.templates.default = 'plotly_dark'
fig = go.Figure()

# iterate through tickers and graphs each with line charts
tickers = list(df['ticker'].unique())
for ticker in tickers:      
    # Add the wave trace
    xdf = df[df['ticker'] == ticker][['date', 'ticker', 'current_shares', 'adj_close']]
    fig.add_trace(go.Scatter(x=xdf['date'], y=(xdf['current_shares'] * xdf['adj_close']), mode='lines', name=ticker))

fig.show()

fig2 = go.Figure()
# adding portfolio value
fig2.add_trace(go.Scatter(x=df['date'], y=df['portfolio_value'], mode='lines', name='Portfolio Value'))
fig2.show()

## Sandbox

A sandbox for testing values

In [10]:
# display(df['ticker'].value_counts())

# display(df[['year', 'ticker', 'volume']].groupby(['year', 'ticker']).agg(['count']))

# display(df[df['year'].isin(list(range(2017, 2023)))]['ticker'].value_counts())
# display(df[df['year'].isin(list(range(2017, 2023)))].shape)


display(df['ticker'].unique())
# display the first date where each ticker is being reported
print(df[['ticker', 'date']].groupby('ticker').agg(['min']).to_string())

array(['FB', 'TWTR', 'ETSY', 'SNAP', 'PINS'], dtype=object)

             date
              min
ticker           
ETSY   2015-04-16
FB     2012-05-18
PINS   2019-04-18
SNAP   2017-03-02
TWTR   2013-11-07
